In [ ]:
#Quality control using trim_galore
#conda create -n trimgalore bioconda -c trimgalore
trim_galore --illumina --paired -o fq_files file1_1.fq file1_2.fq

#for many samples
#!/bin/bash
for i in *_1.fq.gz;
do
        name=${i%_*}
        trim_galore --illumina --paired ${name}_1.fq.gz ${name}_2.fq.gz -o fq_files
done

#for manual adapters
mkdir trimmed_files
for i in *_1.fastq.gz;
do
        name=${i%_*}
        #echo ${name}_1.fastq.gz
        trim_galore --paired \
        -a 'AGATCGGAAGAGCACACGTCTGAACTCCAGTCA' -a2 'AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT' -o ../trimmed_files ${name}_1.fastq.gz ${name}_2.fastq.gz
done


In [ ]:
#Mapping/alignment 
conda install -c bioconda bwa using BWA

#indexing the reference genome
bwa index GCA_021130815.1_PanTigT.MC.v3_genomic.fna

#alignment
#!/bin/bash
for i in *trimmed_files;
do
    name=${i%_*}
    bwa mem ref.fna ${name}_1 ${name}_2 > sam.files
done



In [ ]:
#Removing duplicates
#!/bin/bash
for i in *_sorted.bam;
do
        name=${i%_*}
        #echo ${name}_sorted.bam
        #echo ${name}_rmdup.bam
        samtools rmdup ${name}_sorted.bam  ${name}_rmdup.bam
done



In [ ]:
#sorting
for i in *sam.files;
do
    name=${i%_*}]
    samtools view -q 30 -u ${name}.sam | samtools sort -0 ${name}_sorted.bam
done

In [ ]:
#variant calling 
#indexing the reference genome
samtools faidx GCA_021130815.1_PanTigT.MC.v3_genomic.fna
#indexing the bam files
samtools index ${name}_rmdup.bam

#variant calling using strelka

wget https://github.com/Illumina/strelka/releases/download/v2.9.10/strelka-2.9.10.centos6_x86_64.tar.bz2
tar -xvzf strelka*
tar -xvjf strelka*
bash strelka-2.9.2.centos6_x86_64/bin/runStrelkaSomaticWorkflowDemo.bash
 mv strelka-2.9.10.centos6_x86_64 strelka
rm strelka*.bz2
cd strelka
bash ./bin/runStrelkaSomaticWorkflowDemo.bash
conda create -n strelka python=2.7
conda activate strelka
ls bin
bash ./bin/runStrelkaGermlineWorkflowDemo.bash
rm -rf strelkaGermlineDemoAnalysis
    
nano configure_strelka.sh
    configureStrelkaGermlineWorkflow.py \
    --bam BEN_NW10_rmdup.bam \
    --bam BEN_NW12_rmdup.bam \
    --bam BEN_SI9_rmdup.bam \
    --bam BEN_SI18_rmdup.bam \
    --bam BEN_NW13_rmdup.bam \
    --bam BEN_SI19_rmdup.bam \
    --bam BEN_CI18_rmdup.bam \
    --bam BEN_CI16_rmdup.bam \
    --referenceFasta /home/osboxes/GCA_021130815.1_PanTigT.MC.v3_genomic.fna \
    --runDir .
python runWorkflow.py


In [ ]:
#SNPs filtering
#Filtering the SNPS to remove indels, minimum genotype quality 20, minimum quality 20, remove NOT PASS, minor allelic count 3,
#HWE 0.05,  
vcftools --gzvcf wg_dhole_parallel_va_Chr0.recode.vcf.gz --remove-indels --minQ 20 --minGQ 20 --remove-filtered-all /
--mac 3 --hwe 0.05 --recode --out wg_variants.filtered1

#filter based on loci missing in 80%, 70%, 60%, 50,40 and 30% individual
vcftools --vcf --max-missing 0.6 --out 60%_wg_variants.filtered1.recode.vcf.recode.vcf


In [ ]:
#Genetic Diversity
#heterozigosity
vcftools --vcf 60%_wg_variants.filtered1.recode.vcf.recode.vcf --het --out het_60%_variants.filtered1.recode.vcf

#CALCULATING het from o(hom) and N_sites
less het_60%_variants.filtered1.recode.vcf.het | grep -v "INDV" | awk '{print $1 "\t"(($4-$2)*1000000/120912629)}' | less

#calculating pairwise nucleotide diversity
Create different files for different populations and add individuals in that population

vcftools --vcf 60%_wg_variants.filtered1.recode.vcf.recode.vcf --keep pop1.indv --site-pi --out /
60%_wg_variants.filtered1.recode.vcf.pop1.indv1




In [ ]:
#Population Structure
vcftools --vcf 60%_wg_variants.filtered1.recode.vcf.recode.vcf.genome --plink --out wg_dhole_paralle

plink --file 60%_wg_variants.filtered1.recode.vcf.recode.vcf.genome --make-bed --out 60%_wg_variants.filtered1.recode.vcf.recode.vcf.genome

#Genetic relatedness

plink --vcf 60%_wg_variants.filtered1.recode.vcf.recode.vcf --genome --out 60%_wg_variants.filtered1.recode.vcf.recode.vcf --double-id --allow-extra-chr
less -S 60%_wg_variants.filtered1.recode.vcf.recode.vcf.genome | awk '{print $1"\t"$3"\t"$10}' | sort -k 3 | grep -v "cuonTH3" | less

#PCA
plink --vcf 60%_wg_variants.filtered1.recode.vcf.recode.vcf --pca --out pca_output

#admixture
conda activate admixture

admixture  2

for K in {1..5}; do admixture --cv wg_dhole_parallel_va_Chr0_minQ20_minGQ20_rmvIndel0_minQ20_minGQ20_rmvIndels_PASSonly_mac3_hwe0.05_mm0.6.bed ${K} | tee log${K}.out; done


In [ ]:
#Functional  diversity
conda create -n VEP -c bioconda ensembl-vep
gunzip Tiger_GNE.aed.0.6.gff.gz
grep -v "#" Tiger_GNE.aed.0.6.gff | sort -k1,1 -k4,4n -k5,5n -t$'\t' | bgzip -c > Tiger_GNE.aed.0.6.VEP.gff.gz
#index the gff 
tabix -p gff Tiger_GNE.aed.0.6.VEP.gff.gz

#replace the chromosomes names in the gff file to match the annotation file
sed -i 's/A1/CM037648.1/g;s/A2/CM037649.1/g; s/A3/CM037650.1/g; s/B1/CM037651.1/g; s/B2/CM037652.1/g; s/B3/CM037653.1/g; s/B4/CM037654.1/g; s/C1/CM037655.1/g; s/C2/CM037656.1/g; s/D1/CM037657.1/g; s/D2/CM037658.1/g; s/D3/CM037659.1/g; s/D4/CM037660.1/g; s/E1/CM037661.1/g; s/E2/CM037662.1/g; s/E3/CM037663.1/g; s/F1/CM037664.1/g; s/F2/CM037665.1/g' Tiger_GNE.aed.0.6.gff
conda activate VEP
vep -i variants_minQ30_rmvIndels_PASSonly.recode.vcf --format vcf -o variants_minQ20_rmvIndels_PASSonly.vep --gff Tiger_GNE.aed.0.6.VEP.gff.gz --fasta GCA_021130815.1_PanTigT.MC.v3_genomic.fna --variant class

In [ ]:
#Linkage disequilibrium
#Calculating LD for each population(1..4)
vcftools --vcf 60%_wg_variants.filtered1.recode.vcf.recode.vcf --keep pop1.indv --out 60%_wg_variants.filtered1.recode.vcf_pop1.indv1 \--recode
plink --vcf 60%_wg_variants.filtered1.recode.vcf_pop1.indv1.recode.vcf --r2 --out 60%_wg_variants.filtered1.recode.vcf_pop1.indv1.recode.vcf --allow-extra-chr
less  60%_wg_variants.filtered1.recode.vcf_pop1.indv1.recode.vcf.ld

conda create -n py2 python=2.7 anaconda
conda activate py2
gzip 60%_wg_variants.filtered1.recode.vcf_pop1.indv1.recode.vcf.ld
python ld_decay_calc.py -i 60%_wg_variants.filtered1.recode.vcf_pop1.indv1.recode.vcf.ld.gz -o 60%_wg_variants.filtered1.recode.vcf_pop1.indv1.recode.vcf.Id.decay
less 60%_wg_variants.filtered1.recode.vcf_pop1.indv1.recode.vcf.Id.decay.ld_decay
# plot the LD decay plot